<img src="https://raw.githubusercontent.com/ashlitaylor/ashlitaylor.github.io/master/images/pagerank.jpg" height="600" width="400">
<header> 
    <h2 align="center"> Scalable Single-Machine PageRank on 70M edge graph </h2>
    <h6 align="center"> PageRank algorithm implementation </h6>
</header>

### Background
PageRank, named after Google co-founder Larry Page, is an algorithm that measures webpage importance. According to Google:
>"PageRank works by counting the number and quality of links to a page to determine a rough estimate of how important the website is. The underlying assumption is that more important websites are likely to receive more links from other websites"

The PageRank algorithm I implemented can scale to graph datasets with as many as billions of edges. Ordinarily, running such an algorithm on large datasets would require the use of computer clusters (e.g. Spark, Hadoop), however I implemented this algorithm using my computer's virtual memory. 

#### Virtual Memory
The main idea is to place the dataset in your computer’s (unlimited) virtual memory, as it is often too big to fit in the RAM. When running algorithms on the dataset (e.g., PageRank), the operating system will automatically decide when to load the necessary data (subset of whole dataset) into RAM. See <a href = "http://poloclub.gatech.edu/mmap/">here</a> for more details on research on virtual memory mapping. 

The mapping for this algorithm is done using Python's <a href = "https://docs.python.org/3/library/mmap.html">mmap</a> and <a href = "https://docs.python.org/2/library/struct.html">struct</a> modules, and I used PyPy to expedite the packing and unpacking functionality of Python. 

I used this PageRank algorithm on the <a href = "https://snap.stanford.edu/data/soc-LiveJournal1.html">LiveJournal</a> graph, which contains almost 70 million edges. The algorithm outputs the 10 nodes with the highest PageRank scores. I generated output using 10 iterations, 25 iterations, and 50 iterations. 

#### Motivation
The motivation for this work was an assignment that explored how to use my computer’s virtual memory to implement the PageRank algorithm that will scale to graph datasets with as many as billions of edges using a single computer (e.g. my laptop). 

Below is code for the PageRank algorithm (power iteration).

Here, the binary index_file contains the mapping of each node's ID to its degree.
The node IDs range from 0 to max_node_id.
Hence, the index_file contains (max_node_id + 1) pairs of values,
each of which is of 'int' C type in little endian byte order.
The index_file is memory-mapped into the index_map object.

The binary edge_file contains the edges in the (source ID, target ID) format.
Hence, the edge_file contains edge_count pairs of values,
each of which is of 'int' C type in big endian byte order.
The edge_file is memory-mapped into the edge_map object.

My task was to determine the correct parameters needed to
(1) initialize the memory-mapped objects (index_map and edge_map),
(2) unpack the source and target IDs from the edge_map, and
(3) upack the source ID and source degree from the index_map.

This code assumes that the node IDs start from 0 and are contiguous up to max_node_id.
This algorithm returns the same scores as popular graph analysis libraries like NetworkX.


In [ ]:
def pagerank(index_file, edge_file, max_node_id, edge_count, damping_factor=0.85, iterations=10):
    index_map = mmap.mmap(
        index_file.fileno(),
        length= (max_node_id+1)*8,  
        access=mmap.ACCESS_READ)

    edge_map = mmap.mmap(
        edge_file.fileno(),
        length= edge_count * 8,  
        access=mmap.ACCESS_READ)

    scores = [1.0 / (max_node_id + 1)] * (max_node_id + 1)

    start_time = time.time()

    for it in range(iterations):
        new_scores = [0.0] * (max_node_id + 1)

        for i in range(edge_count):
            source, target = unpack(
                '>ii',  
                edge_map[i * 8: i * 8 + 8])  

            source_degree = unpack(
                '<ii',  
                index_map[source * 8: source * 8 + 8])[1]  

            new_scores[target] += damping_factor * scores[source] / source_degree

        min_pr = (1 - damping_factor) / (max_node_id + 1)
        new_scores = [min_pr + item for item in new_scores]
        scores = new_scores

        print ("Completed {0}/{1} iterations. {2} seconds elapsed." \
            .format(it + 1, iterations, time.time() - start_time))

    print ()

    return scores